# Profiling results

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from functools import wraps
from time import time
import scipy.sparse

In [2]:
def timing(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print("Elapsed time: {}".format(end-start))
        return result
    return wrapper

# Euclidean distance between two sets

In [3]:
np.random.seed(5566)
N = 1000
M = 100
D = 50

In [4]:
A = np.random.uniform(low=-1, high=1, size=(N, D))
B = np.random.uniform(low=-1, high=1, size=(N, D))

In [5]:
# def getDistanceLoop(A, B):
#     N = A.shape[0]
#     M = A.shape[0]
    
#     C = np.zeros((N, M))
    
#     for n in range(N):
#         for m in range(M):
#             C[n, m] = np.sqrt(np.sum((A[n, :] - B[m, :])**2))
            
#     return C

# def getDistanceVec(A, B):
#     N = A.shape[0]
#     M = B.shape[0]
#     D = A.shape[1]
    
#     return np.sqrt(A**2@np.ones([D, M]) + np.ones([N, D])@B.T**2 - 2*A@B.T)

In [6]:
@timing
def getDistanceLoop(A, B):
    N = A.shape[0]
    M = A.shape[0]
    
    C = np.zeros((N, M))
    
    for n in range(N):
        for m in range(M):
            C[n, m] = np.sqrt(np.sum((A[n, :] - B[m, :])**2))
            
    return C

In [7]:
@timing
def getDistanceVec(A, B):
    N = A.shape[0]
    M = B.shape[0]
    D = A.shape[1]
    
    return np.sqrt(A**2@np.ones([D, M]) + np.ones([N, D])@B.T**2 - 2*A@B.T)


In [8]:
C_loop = getDistanceLoop(A, B)
C_vec = getDistanceVec(A,B)

Elapsed time: 5.721044540405273
Elapsed time: 0.08269190788269043


In [9]:
np.allclose(C_loop, C_vec)

True

# Cluster centers

In [10]:
np.random.seed(5566)
N = 1000
D = 50
K = 100
cluster_assign = np.random.choice(K, N)
I = cluster_assign
J = np.arange(N)
V = np.ones(N)
E = scipy.sparse.coo_matrix((V,(I,J)),shape=(K, N)).toarray()

X = np.random.uniform(low=-1, high=1, size=(N, D))

In [11]:
# def getCenters1(X, E):
#     K = E.shape[0]
#     D = X.shape[1]
#     C = np.zeros([K, D])
    
#     for k in range(K):
#         member_index = np.where(E[k, :]==1)[0]
#         C[k, :] = np.mean(X[member_index, :], axis=0)
#     return C

# def getCenters2(X, E):
#     return E@X/np.sum(E, axis=1, keepdims=True)

In [12]:
@timing
def getCenters1(X, E):
    K = E.shape[0]
    D = X.shape[1]
    C = np.zeros([K, D])
    
    for k in range(K):
        member_index = np.where(E[k, :]==1)[0]
        C[k, :] = np.mean(X[member_index, :], axis=0)
    return C

In [13]:
@timing
def getCenters2(X, E):
    return E@X/np.sum(E, axis=1, keepdims=True)

In [14]:
C_1 = getCenters1(X, E)
C_2 = getCenters2(X, E)
np.allclose(C_1, C_2)

Elapsed time: 0.002004384994506836
Elapsed time: 0.0020363330841064453


True

# K means

In [15]:
N = 1000
D = 50
K = 100
X = np.random.uniform(low=-1, high=1, size=(N, D))

def kmeans(X, K):
    # for sparse matrix
    J = np.arange(N)
    V = np.ones(N)

    cluster_assign = np.random.choice(K, N)
    prev_assign = np.zeros(cluster_assign.shape, dtype=np.int32)

    while any(cluster_assign != prev_assign):
        prev_assign = cluster_assign
        E = scipy.sparse.coo_matrix((V,(prev_assign,J)),shape=(K, N)).toarray() # indicator matrix
        centers = E@X/np.sum(E, axis=1, keepdims=True) # K x D
        dist = centers**2@np.ones([D, N]) - 2*centers@X.T # KxN
        cluster_assign = np.argmin(dist, axis=0)
    return cluster_assign

# loggausspdf

In [16]:
from scipy.stats import multivariate_normal

In [17]:
N = 1000
D = 10

X = np.random.uniform(low=-1, high=1, size=(N, D))

mu = np.mean(X, axis=0)
Sigma = np.cov(X.T)

In [28]:
def loggausspdf(X, mu, Sigma):
    D = X.shape[1]
    x_tilde = X-mu
    U = np.linalg.cholesky(Sigma)
    v = np.linalg.solve(U.T, x_tilde.T)
    Q = np.sum(v**2, axis=0)
    constant = D*np.log(2*np.pi)+2*np.sum(np.log(np.diag(U)))

    return -(Q+constant)/2

In [21]:
ll = loggausspdf(X, mu, Sigma)
ll2 =  multivariate_normal.logpdf(X, mean=mu, cov=Sigma)

In [23]:
ll

array([-12.79154211, -11.99337849, -13.19956449, -12.21981522,
       -13.15156482, -13.12272836, -10.97246923, -10.06962386,
       -11.19526187, -10.25019603, -13.79460167,  -9.28353097,
       -10.69776119, -12.06535932, -11.61020237,  -9.31616153,
       -14.58866126,  -8.54792127, -12.07102178, -11.65341984,
        -9.48634184, -14.67068619, -11.0161536 , -12.09870957,
       -11.38530265, -11.76118757, -10.38895621, -11.91737337,
       -10.4150218 , -11.44260369, -11.24883131, -13.23271645,
       -10.43974963, -10.30148282, -10.15009798, -11.47959921,
       -10.93826172,  -9.61290757, -10.03755604, -13.43282168,
        -9.73139855,  -9.17608373,  -8.39013464,  -9.74426483,
        -9.05941843, -11.32387206, -11.76441709, -11.678793  ,
       -13.28068062,  -9.02145909, -13.41089714, -11.49964538,
        -9.95281586, -11.88555866,  -8.23405739, -15.77942855,
       -12.55091146,  -9.35557983, -12.21687921, -12.00985681,
       -12.19461093, -11.05095316, -11.52761267, -11.19

In [22]:
ll2

array([-10.04381281,  -9.22857961, -10.42569705,  -9.43930636,
       -10.3861337 , -10.36989772,  -8.19674396,  -7.30023093,
        -8.43731478,  -7.48110728, -11.00815084,  -6.50646471,
        -7.93344082,  -9.31638844,  -8.8315997 ,  -6.54678263,
       -11.82069078,  -5.77119914,  -9.29790878,  -8.88251823,
        -6.71586387, -11.90106305,  -8.23407074,  -9.33532457,
        -8.61989217,  -8.98787225,  -7.61769612,  -9.13856222,
        -7.63872555,  -8.65374291,  -8.46795607, -10.44988588,
        -7.67511307,  -7.53685656,  -7.37102885,  -8.69644032,
        -8.16080878,  -6.85266738,  -7.26414563, -10.66292092,
        -6.94972929,  -6.40637812,  -5.62398852,  -6.9755977 ,
        -6.28525249,  -8.56323271,  -9.01452248,  -8.89342021,
       -10.50440442,  -6.24841858, -10.64838216,  -8.74129421,
        -7.17824159,  -9.1323123 ,  -5.46265977, -13.04571011,
        -9.7754185 ,  -6.58086821,  -9.42530014,  -9.23661979,
        -9.42038863,  -8.29217686,  -8.77277419,  -8.41